<a href="https://colab.research.google.com/github/yeoun9/torchpm/blob/main/examples/example_covariate_searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/yeoun9/torchpm.git
! pip install sympytorch

In [2]:
from torchpm import *
from torchpm.data import CSVDataset
from torchpm.parameter import *
from torchpm import covariate
import torch as tc
import numpy as np
import matplotlib.pyplot as plt

# Model Building

In [5]:
dataset_file_path = 'https://raw.githubusercontent.com/yeoun9/torchpm/main/examples/THEO_cov_searching.csv'
dataset_np = np.loadtxt(dataset_file_path, delimiter=',', dtype=np.float32, skiprows=1)
device = tc.device("cuda:0" if tc.cuda.is_available() else "cpu")
column_names = ['ID', 'AMT', 'TIME', 'DV', 'CMT', "MDV", "RATE", 'BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']
dataset = CSVDataset(dataset_np, column_names, device)

class BasementModel(predfunction.PredictionFunctionByTime) :
    
    def _set_estimated_parameters(self):
        self.theta_0 = Theta(0., 1.5, 10.)
        self.theta_1 = Theta(0., 30., 100.)
        self.theta_2 = Theta(0, 0.08, 1)

        self.eta_0 = Eta()
        self.eta_1 = Eta()
        self.eta_2 = Eta()

        self.eps_0 = Eps()
        self.eps_1 = Eps()

        self.gut_model = linearode.Comp1GutModelFunction()
    
    def _calculate_parameters(self, covariates):
        covariates['k_a'] = self.theta_0()*tc.exp(self.eta_0())
        covariates['v'] = self.theta_1()*tc.exp(self.eta_1())#*para['BWT']/70
        covariates['k_e'] = self.theta_2()*tc.exp(self.eta_2())
        covariates['AMT'] = tc.tensor(320., device=self.dataset.device)

    def _calculate_preds(self, t, p):
        dose = p['AMT'][0]
        k_a = p['k_a']
        v = p['v']
        k_e = p['k_e']
        comps = self.gut_model(t, k_a, k_e, dose)
        return comps[1]/v
        
    def _calculate_error(self, y_pred, p):
        p['v_v'] = p['v'] 
        return y_pred +  y_pred * self.eps_0() + self.eps_1()

dependent_parameter_names = ['k_a', 'v', 'k_e']
dependent_parameter_initial_values = [[0,1.4901,2],[30,32.4667,34],[0,0.08,0.1]]
independent_parameter_names = ['BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']

searcher = covariate.DeepCovariateSearching(dataset=dataset,
                                BaseModel=BasementModel,
                                dependent_parameter_names=dependent_parameter_names,
                                independent_parameter_names=independent_parameter_names,
                                dependent_parameter_initial_values=dependent_parameter_initial_values,
                                eps_names=['eps_0', 'eps_1'],
                                omega = Omega([0.4397,
                                                0.0575,  0.0198, 
                                                -0.0069,  0.0116,  0.0205], False, requires_grads=True),
                                sigma = Sigma([[0.0177], [0.0762]], [True, True], requires_grads=[True, True]))
result = searcher.run(tolerance_grad=0.5e-2, tolerance_change=0.5e-2)

running_time :  1.2835676670074463 	 total_loss: tensor(8841.1572, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  2.525139331817627 	 total_loss: tensor(3159.0715, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.812077283859253 	 total_loss: tensor(744.2010, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.064607381820679 	 total_loss: tensor(418.9419, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  6.288004636764526 	 total_loss: tensor(328.8091, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  7.53707218170166 	 total_loss: tensor(293.1155, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  8.799615621566772 	 total_loss: tensor(250.8653, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  10.03028392791748 	 total_loss: tensor(203.5450, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  11.291890144348145 	 total_loss: tensor(179.6786, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  12.556850910186768 	 total_

In [6]:
print('selected covariates \n', result['selected covariates'])

for record in result['history'] :
    print(record)


selected covariates 
 ['BWT', 'BWT-0.5+0.5']
{'loss': 74.38174438476562, 'removed covariates': [], 'selected': True}
{'loss': 75.73906707763672, 'loss difference:': 1.3573226928710938, 'removed covariates': ['fixed'], 'selected': False}
{'loss': 75.49971008300781, 'loss difference:': -0.23935699462890625, 'removed covariates': ['fixed', 'rand-1+1'], 'selected': False}
{'loss': 78.40115356445312, 'loss difference:': 2.9014434814453125, 'removed covariates': ['fixed', 'rand-1+1', 'norm(0,1)'], 'selected': False}
